In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, root_mean_squared_error,r2_score
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from pytorch_tabnet.tab_model import TabNetRegressor

from hyperopt import fmin, tpe, hp, Trials

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pre_processing import preprocess_data

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
def load_data():
    dataset = pd.read_excel(r"globalterrorismdb_2021Jan-June_1222dist.xlsx") # 2021-2021 June
    return dataset

## Data Loading and Preprocessing

In [3]:
dataset = load_data()
X, y, dataset = preprocess_data(dataset)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dataset.head()

,success,gname_freq,city_freq,country_freq,attacktype1_score,targtype1_score,weaptype1_score,gname_score,country_score,city_score,...,nkill_likelihood_score,region_3,region_5,region_6,region_8,region_9,region_10,region_11,region_12,nkill
0,1.0,0.228571,0.066667,0.167228,0.875,1.000000,0.857143,0.977778,0.948718,0.123077,...,0.589934,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
1,1.0,0.121429,0.033333,0.096521,0.500,0.842105,0.857143,0.911111,0.871795,0.030769,...,0.347526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.000000,0.033333,0.159371,0.875,0.894737,0.857143,0.000000,0.641026,0.015385,...,0.477050,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.121429,0.033333,0.096521,0.875,0.894737,0.857143,0.911111,0.871795,0.092308,...,0.503175,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0
4,1.0,0.000000,0.033333,0.093154,0.875,1.000000,0.857143,0.022222,0.743590,0.030769,...,0.543654,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## 2) TabNet
## Hyperparameter Tuning and Model Training

In [4]:
def objective(params):
    # Extract hyperparameters
    n_d = int(params['n_d'])
    n_a = int(params['n_a'])
    n_steps = int(params['n_steps'])
    gamma = params['gamma']
    lambda_sparse = params['lambda_sparse']
    lr = params['lr']
    weight_decay = params['weight_decay']
    
    # Initialize TabNet model with current hyperparameters
    tabNetModel = TabNetRegressor(
        n_d=n_d,            
        n_a=n_a,             
        n_steps=n_steps,     
        gamma=gamma,         
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.AdamW,
        optimizer_params=dict(lr=lr, weight_decay=weight_decay),
        scheduler_params={"step_size":8, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type="entmax",
    )
    
    # Train the model and get validation loss (e.g., RMSE)
    tabNetModel.fit(
        X_train=X_train, 
        y_train=y_train.reshape(-1, 1),
        eval_set=[(X_test, y_test.reshape(-1, 1))],
        eval_name=['val'],
        eval_metric=['rmse'],
        max_epochs=40,
        patience=15,
        batch_size=64,
        virtual_batch_size=32,
        num_workers=3,
        drop_last=True
    )
    
    # Extract validation RMSE (or any other metric you prefer)
    val_rmse = tabNetModel.history.history['val_rmse'][-1]  # Get the last RMSE value
    return val_rmse

# Define the search space
space = {
    'n_d': hp.quniform('n_d', 8, 64, 16),          # Number of decision units
    'n_a': hp.quniform('n_a', 8, 64, 16),          # Number of attention units
    'n_steps': hp.quniform('n_steps', 3, 10, 2),   # Number of steps in TabNet
    'gamma': hp.uniform('gamma', 1.0, 2.0),        # Sparsity controlling factor
    'lambda_sparse': hp.loguniform('lambda_sparse', -5, -1),  # Sparsity regularization term
    'lr': hp.loguniform('lr', -5, -1),             # Learning rate (log scale)
    'weight_decay': hp.loguniform('weight_decay', -5, -1)  # Weight decay (log scale)
}

# Initialize Trials object to store results
trials = Trials()

# Run the optimization with Hyperopt
best = fmin(
    fn=objective,             # Objective function
    space=space,              # Search space
    algo=tpe.suggest,         # Optimization algorithm (Tree of Parzen Estimators)
    max_evals=15,             # Number of evaluations
    trials=trials             # Store trials
)

# Print the best hyperparameters found
print("Best hyperparameters:", best)


  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

c:\Users\rma81\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")



epoch 0  | loss: 44.52012| val_rmse: 7.11181 |  0:00:21s
epoch 1  | loss: 40.02954| val_rmse: 7.03802 |  0:00:36s
epoch 2  | loss: 35.6399 | val_rmse: 6.27565 |  0:00:54s
epoch 3  | loss: 30.95694| val_rmse: 6.79133 |  0:01:10s
epoch 4  | loss: 29.27898| val_rmse: 5.34931 |  0:01:27s
epoch 5  | loss: 26.85812| val_rmse: 6.86841 |  0:01:46s
epoch 6  | loss: 24.44915| val_rmse: 6.72198 |  0:02:02s
epoch 7  | loss: 24.74707| val_rmse: 4.88494 |  0:02:19s
epoch 8  | loss: 24.47092| val_rmse: 5.28352 |  0:02:37s
epoch 9  | loss: 22.90403| val_rmse: 5.42474 |  0:02:55s
epoch 10 | loss: 22.11533| val_rmse: 6.45587 |  0:03:13s
epoch 11 | loss: 22.78002| val_rmse: 5.34018 |  0:03:28s
epoch 12 | loss: 20.97734| val_rmse: 3.91207 |  0:03:44s
epoch 13 | loss: 18.74327| val_rmse: 3.49495 |  0:04:00s
epoch 14 | loss: 19.58238| val_rmse: 3.36599 |  0:04:19s
epoch 15 | loss: 18.98585| val_rmse: 3.00902 |  0:04:38s
epoch 16 | loss: 20.61077| val_rmse: 2.97067 |  0:04:58s
epoch 17 | loss: 19.15022| val_

c:\Users\rma81\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



  7%|▋         | 1/15 [11:06<2:35:37, 666.94s/trial, best loss: 3.907758157350823]

...

 93%|█████████▎| 14/15 [2:19:52<09:16, 556.28s/trial, best loss: 3.184476332710138]

c:\Users\rma81\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")



epoch 0  | loss: 45.58073| val_rmse: 7.1364  |  0:00:13s                           
epoch 1  | loss: 37.74866| val_rmse: 6.50126 |  0:00:25s                           
epoch 2  | loss: 31.62779| val_rmse: 6.75511 |  0:00:38s                           
epoch 3  | loss: 30.85029| val_rmse: 6.48796 |  0:00:51s                           
epoch 4  | loss: 26.15702| val_rmse: 6.54951 |  0:01:04s                           
epoch 5  | loss: 24.53813| val_rmse: 7.20062 |  0:01:17s                           
epoch 6  | loss: 23.09699| val_rmse: 6.05022 |  0:01:30s                           
epoch 7  | loss: 22.73302| val_rmse: 5.73868 |  0:01:43s                           
epoch 8  | loss: 22.79028| val_rmse: 6.20147 |  0:01:56s                           
epoch 9  | loss: 21.28281| val_rmse: 5.99148 |  0:02:09s                           
epoch 10 | loss: 22.45807| val_rmse: 5.34003 |  0:02:22s                           
epoch 11 | loss: 21.40658| val_rmse: 5.36677 |  0:02:35s                    

c:\Users\rma81\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



100%|██████████| 15/15 [2:28:36<00:00, 594.46s/trial, best loss: 3.184476332710138]
Best hyperparameters: {'gamma': np.float64(1.5494371504360784), 'lambda_sparse': np.float64(0.021527849243156594), 'lr': np.float64(0.037088099470451996), 'n_a': np.float64(48.0), 'n_d': np.float64(32.0), 'n_steps': np.float64(8.0), 'weight_decay': np.float64(0.16622206018645966)}
